In [ ]:
import os
import datetime
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build



In [ ]:

# Google Calendar API scope
SCOPES = ['https://www.googleapis.com/auth/calendar.events']

def schedule_service(date_time: str, description: str):
    """Schedules a service appointment in Google Calendar."""

    creds = None

    # Check for existing saved token
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    else:
        # First-time login: start local server flow
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
        # Save token for next time
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    # Build the calendar service
    service = build('calendar', 'v3', credentials=creds)

    # Define event details
    event = {
        'summary': 'Car Service Appointment',
        'description': description,
        'start': {
            'dateTime': date_time,
            'timeZone': 'Australia/Sydney',  # Adjust timezone as needed
        },
        'end': {
            'dateTime': (datetime.datetime.fromisoformat(date_time) + datetime.timedelta(hours=1)).isoformat(),
            'timeZone': 'Australia/Sydney',
        },
    }

    # Create event in primary calendar
    event = service.events().insert(calendarId='primary', body=event).execute()

    print(f"✅ Event created: {event.get('htmlLink')}")


# 🧪 TEST IT
if __name__ == '__main__':
    # Format: YYYY-MM-DDTHH:MM:SS
    test_datetime = "2025-05-01T14:00:00"
    test_description = "Oil change and 20-point inspection"

    schedule_service(test_datetime, test_description)